# Integrate TigerGraph with LangChain with pyTigerGraph

LangChain is a Python library that integrates Large Language Models (LLMs) such as OpenAI's GPT models and Google's BARD models, among others. There are many benefits to linking a natural language interface to graph databases such as TigerGraph, as models can be grounded in their generation based off of the data the database provides. This reduces the chances of these model's responses to be inaccurate, known as *hallucination*.

In this demo, we are going to integrate LangChain with the Python driver of TigerGraph, pyTigerGraph. We will build up various LangChain tools in order for the LLMs to best extract the information from the database.

## Install Dependencies
First, we are going to install the dependencies. This consists of LangChain and OpenAI's package.

In [1]:
!pip install langchain openai

## Configure Connection to OpenAI and TigerGraph Database

Here, we are going to set our OpenAI API key as an environment variable for use by the LangChain library. **NOTE: You must provide your own OpenAI API key in the cell below.** Additionally, we are going to connect to our TigerGraph database, which has the OGB_MAG citation network loaded into it.

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "OPEN_AI_API_KEY_HERE"

In [3]:
from pyTigerGraph import TigerGraphConnection
from pyTigerGraph.datasets import Datasets
import json

# Read in DB configs
with open('./config.json', "r") as config_file:
    config = json.load(config_file)
    
conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

In [ ]:
dataset = Datasets("OGB_MAG")
conn.ingestDataset(dataset, getToken=config["getToken"])

In [4]:
# Cleanup from previous executions of the notebook

conn.gsql("""USE GRAPH OGB_MAG
             DROP QUERY get_papers_of_author
             DROP QUERY author_at_institutions
             DROP QUERY tg_pagerank""")

"Using graph 'OGB_MAG'\nSuccessfully dropped queries on the graph 'OGB_MAG': [get_papers_of_author].\nSuccessfully dropped queries on the graph 'OGB_MAG': [author_at_institutions].\nSuccessfully dropped queries on the graph 'OGB_MAG': [tg_pagerank]."

In [5]:
conn.getVertexCount("*")

{'FieldOfStudy': 59965,
 'Institution': 8740,
 'Paper': 736389,
 'Author': 1134649}

## Creating the LangChain Tools (Part 1)

We are going to iterate through the building and testing of LangChain tools, progressively adding more functionality to the agent. First, we import the needed LangChain dependencies. Then, we are going to build three core tools: **MapQuestionToSchema**, **GenerateFunction**, and **ExecuteFunction**.

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.llms.base import LLM
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.tools.base import ToolException
from langchain.chains import LLMChain
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import Optional, Type

### Creating the MapQuestionToSchema Tool
We want to map a user's question in natural language to the specific graph schema. For example, a question such as "How many writers are there?" should translate to "How many Author Vertices are there?", as there is not a schema element for "Writers", but there is one for "Author".

We implement this via another call to a large language model, while passing the original question along with the schema elements retrieved from the graph connection. Specifically, we use pyTigerGraph's `getVertexTypes()` and `getEdgeTypes()` functions to gather schema information to be passed to the LLM.

In [7]:
class MapQuestionToSchema(BaseTool):
    name = "MapQuestionToSchema"
    description = "Always run first to map the query to the graph's schema"
    conn: TigerGraphConnection = None
    llm: LLM = None
    handle_tool_error: bool = True
    
    def __init__(self, conn, llm):
        super().__init__()
        self.conn = conn
        self.llm = llm
        
    def _run(self, query: str) -> str:
        """Use the tool."""
        question_restate = """Replace the entites mentioned in the question to one of these choices: {vertices}.
                      Replace the relationships mentioned in the question to one of these choices: {edges}.
                      Generate the complete question with the appropriate replacements. Keep the case of the schema elements the same.
                      If there are no replacements to be made, restate the question.
                      Example: How many universities are there?
                      Response: How many vertices are University Vetexes?
                      Example: What is the schema?
                      Response: What is the schema?
                      Example: How many transactions are there?
                      Response: How many TRANSACTION Edges are there?
                      QUESTION: {question}
                      RESTATED: """

        RESTATE_QUESTION_PROMPT = PromptTemplate(
            template=question_restate, input_variables=["question", "vertices", "edges"]
        )
        
        restate_chain = LLMChain(llm=self.llm, prompt=RESTATE_QUESTION_PROMPT)
        
        restate_q = restate_chain.apply([{"vertices": [x + " Vertex" for x in self.conn.getVertexTypes()], # + [x + " Edge" for x in conn.getEdgeTypes()],
                                          "question": query,
                                          "edges": [x + " Edge" for x in self.conn.getEdgeTypes()]}])[0]["text"]
        return restate_q
    
    async def _arun(self) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
        
    def _handle_error(self, error:ToolException) -> str:
        return  "The following errors occurred during tool execution:" + error.args[0]+ "Please ask for human input if they asked their question correctly."

### Creating the GenerateFunctionTool

We want the LLM agent to be able to generate pyTigerGraph function calls for eventual execution. We are going to once again pass the schema information, the installed queries (even though there are none installed currently), and an example of a built-in function call for pyTigerGraph such as `getVertexCount()`.

This tool also utilizes pyTigerGraph to retrieve the installed queries on the graph and their parameters. This information is passed to the prompt asking the LLM to generate the appropriate function call.

In [8]:
class GenerateFunction(BaseTool):
    name = "GenerateFunction"
    description = "Generate a pyTigerGraph function call"
    conn: TigerGraphConnection = None
    llm: LLM = None
    handle_tool_error: bool =True
    
    def __init__(self, conn, llm):
        super().__init__()
        self.conn = conn
        self.llm = llm
    
    def _run(self, question: str) -> str:
        prompt_template = """Use the vertex types, edge types to write the pyTigerGraph function call to answer the question using a pyTigerGraph connection.
                            For example, if a count of vertices is asked for, use getVertexCount().
                            If multiple entites are referred to in the question, use runInstalledQuery() when necessary.
                            If there is not a natural answer to the question, ask a human for help.
                            Parameters of queries to be replaced are denoted <INSERT_ID_HERE>.
                            Vertex Types: {vertices}
                            Edge Types: {edges}
                            Queries: {queries}
                            Question: {question}
                            Python Call: conn."""
        
        PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["question", "vertices", "queries", "edges"]
        )
        queries = conn.getInstalledQueries()
        [queries[x]["parameters"].pop("read_committed") for x in queries] #remove read_committed
        inputs = [{"question": question, 
                    "vertices": self.conn.getVertexTypes(), 
                    "edges": self.conn.getEdgeTypes(), 
                    "queries": [{queries[x]["alternative_endpoint"].split("/")[-1]: queries[x]["parameters"]} for x in queries]
                  }]
        
        chain = LLMChain(llm=self.llm, prompt=PROMPT)
        generated = chain.apply(inputs)[0]["text"]
        return generated

    async def _arun(self) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
        
    def _handle_error(error:ToolException) -> str:
        return  "The following errors occurred during tool execution:" + error.args[0]+ "Please ask for human input if they asked their question correctly."

### Creating ExecuteFunction Tool
Once the function has been generated, the ExecuteFunction tool will utilize Python and pyTigerGraph to run the generated function against the TigerGraph database. The tool will return the JSON returned by the TigerGraph REST API endpoint.

In [9]:
class ExecuteFunction(BaseTool):
    name = "ExecuteFunction"
    description = "Execute a pyTigerGraph function"
    conn: TigerGraphConnection = None
    handle_tool_error: bool =True
    
    def __init__(self, conn):
        super().__init__()
        self.conn = conn
        
    def _run(self, function: str) -> str:
        try:
            loc = {}
            exec("res = conn."+function, {"conn": conn}, loc)
            return loc["res"]
        except:
            raise ToolException("The function {} did not execute directly. Please rephrase your question and try again".format(function))
    
    async def _arun(self) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
        
    def _handle_error(error:ToolException) -> str:
        return  "The following errors occurred during tool execution:" + error.args[0]+ "Please ask for human input if they asked their question correctly."

## Initialize LLM and AI Agent

Here, we are going to initialize our OpenAI GPT-3 LLM connection, define our tools, and create our LangChain AI Agent. The Agent is able to execute multiple tasks/tools in a linear fashion until it is satisified to answer the question. In this case, we are going to use the tools defined above to start answering questions.

In [10]:
llm = OpenAI(temperature=0, model_name="text-davinci-003")

In [11]:
tools = [MapQuestionToSchema(conn, llm), GenerateFunction(conn, llm),  ExecuteFunction(conn)]

In [12]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

## Start Asking Questions
We are going to start asking the agent some questions. We are going to start easy and progressively get harder, and observe when things break. After that, we will modify our tool set and add queries to the database to help the agent answer more questions. We will start with basic counts of certain schema elements, then work our way into traversals and analytic queries.

### Some Easy Questions
Here, we ask the agent some basic questions, such as "how many papers are there?" and "how many citations are there?" - notice that the agent usedc the `getEdgeCount()` function of pyTigerGraph despite never specifically being prompted to do so. This shows the LLM's inference power to transfer the prompt example of `getVertexCount()` and use it for edge counting instead. Additionally, the "How many writers are there?" was correctly translated to "How many Author Vertices are there?" to match the graph's schema.

In [13]:
agent.run("How many Papers are there?")



> Entering new AgentExecutor chain...
 I need to find out how many Papers are in the graph
Action: MapQuestionToSchema
Action Input: How many Papers are there?
Observation:  How many vertices are Paper Vertexes?
Thought: I need to generate a function to count the number of Paper Vertexes
Action: GenerateFunction
Action Input: Count the number of Paper Vertexes
Observation: getVertexCount('Paper')
Thought: I need to execute the function to get the count
Action: ExecuteFunction
Action Input: getVertexCount('Paper')
Observation: 736389
Thought: I now know the final answer
Final Answer: There are 736389 Papers.

> Finished chain.


'There are 736389 Papers.'

In [14]:
agent.run("How many citations are there?")



> Entering new AgentExecutor chain...
 I need to find out how many citations are in the graph
Action: MapQuestionToSchema
Action Input: How many citations are there?
Observation:  How many CITES Edges are there?
Thought: I need to generate a function to count the number of CITES edges
Action: GenerateFunction
Action Input: COUNT CITES
Observation: getEdgeCount('CITES')
Thought: I need to execute the function to get the number of CITES edges
Action: ExecuteFunction
Action Input: getEdgeCount('CITES')
Observation: 5416271
Thought: I now know the final answer
Final Answer: There are 5416271 citations.

> Finished chain.


'There are 5416271 citations.'

In [15]:
agent.run("How many writers are there?")



> Entering new AgentExecutor chain...
 I need to find out how many writers there are in the graph.
Action: MapQuestionToSchema
Action Input: How many writers are there?
Observation:  How many vertices are Author Vertexes?
Thought: I need to generate a function to count the number of Author vertices.
Action: GenerateFunction
Action Input: Count the number of Author vertices
Observation: getVertexCount('Author')
Thought: I need to execute the function to get the answer.
Action: ExecuteFunction
Action Input: getVertexCount('Author')
Observation: 1134649
Thought: I now know the final answer.
Final Answer: 1134649 writers.

> Finished chain.


'1134649 writers.'

### Some Hard Questions
Here, we are going to ask some questions that the current prompts and queries installed on the database can't answer. We will start with a simple `getVertexCount()` but with a `where` filter, then move on to traversal and analytic queries. These questions will cause the agent to work really hard, but won't find a solution within the execution limit.

In [16]:
agent.run("How many papers have a y attribute equal to 1?")



> Entering new AgentExecutor chain...
 I need to find all the papers with a y attribute equal to 1.
Action: MapQuestionToSchema
Action Input: How many papers have a y attribute equal to 1?
Observation:  How many Paper Vertexes have a y attribute equal to 1?
Thought: I need to generate a function to find the answer
Action: GenerateFunction
Action Input: Find all Paper Vertexes with a y attribute equal to 1
Observation: runInstalledQuery('Find_Papers_with_y_attribute_equal_to_1', {'y': 1})
Thought: I need to execute the function
Action: ExecuteFunction
Action Input: runInstalledQuery('Find_Papers_with_y_attribute_equal_to_1', {'y': 1})
Observation: The function runInstalledQuery('Find_Papers_with_y_attribute_equal_to_1', {'y': 1}) did not execute directly. Please rephrase your question and try again
Thought: I need to rephrase my question
Action: MapQuestionToSchema
Action Input: How many Paper Vertexes have a y attribute equal to 1?
Observation:  How many Paper Vertexes have an attrib

'Agent stopped due to iteration limit or time limit.'

In [17]:
agent.run("What institutions did author 45 work at?")



> Entering new AgentExecutor chain...
 I need to find the institutions associated with author 45
Action: MapQuestionToSchema
Action Input: What institutions did author 45 work at?
Observation:  What Institutions are IS_AFFILIATED_WITH Edges connected to Author Vertex 45?
Thought: I need to generate a function to get the institutions associated with author 45
Action: GenerateFunction
Action Input: Get the IS_AFFILIATED_WITH Edges connected to Author Vertex 45
Observation: runInstalledQuery('getEdges', {'vertexType': 'Author', 'vertexId': '45', 'edgeType': 'IS_AFFILIATED_WITH'})
Thought: I need to execute the function to get the institutions associated with author 45
Action: ExecuteFunction
Action Input: runInstalledQuery('getEdges', {'vertexType': 'Author', 'vertexId': '45', 'edgeType': 'IS_AFFILIATED_WITH'})
Observation: The function runInstalledQuery('getEdges', {'vertexType': 'Author', 'vertexId': '45', 'edgeType': 'IS_AFFILIATED_WITH'}) did not execute directly. Please rephrase yo

'Agent stopped due to iteration limit or time limit.'

In [18]:
agent.run("By citations, what are the 10 most influential papers?")



> Entering new AgentExecutor chain...
 I need to find the papers with the most citations
Action: MapQuestionToSchema
Action Input: By citations, what are the 10 most influential papers?
Observation:  By CITES Edges, what are the 10 most influential Paper Vertices?
Thought: I need to generate a function to get the most influential papers
Action: GenerateFunction
Action Input: By CITES Edges, what are the 10 most influential Paper Vertices?
Observation: runInstalledQuery('MostInfluentialPapers', {'topK': 10})
Thought: I need to execute the function to get the most influential papers
Action: ExecuteFunction
Action Input: runInstalledQuery('MostInfluentialPapers', {'topK': 10})
Observation: The function runInstalledQuery('MostInfluentialPapers', {'topK': 10}) did not execute directly. Please rephrase your question and try again
Thought: I need to find the papers with the most citations
Action: MapQuestionToSchema
Action Input: By citations, what are the 10 most cited papers?
Observation:

'Agent stopped due to iteration limit or time limit.'

## Add Some Queries to Better Answer Questions

Here we are going to add some queries to the graph to better answer some "domain specific" questions. These queries are quite simple, but provide a template of what more complex queries could take the form of.

In [19]:
conn.gsql("""USE GRAPH OGB_MAG
            CREATE OR REPLACE QUERY author_at_institutions(VERTEX<Author> auth_id) FOR GRAPH OGB_MAG { 
              start = {auth_id};

              res = SELECT i FROM start:s -()- Institution:i;
              PRINT res[res.id];
            }
            INSTALL QUERY author_at_institutions""")

'Using graph \'OGB_MAG\'\nWarning in a transformed block of query author_at_institutions (WARN-2): line 6, col 31\nunsatisfiable pattern _\nSuccessfully created queries: [author_at_institutions].\nStart installing queries, about 1 minute ...\nauthor_at_institutions query: curl -X GET \'https://127.0.0.1:9000/query/OGB_MAG/author_at_institutions?auth_id=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [20]:
conn.gsql("""USE GRAPH OGB_MAG
            CREATE OR REPLACE QUERY get_papers_of_author(VERTEX<Author> auth) FOR GRAPH OGB_MAG { 
              start = {auth};
              res = SELECT p FROM start:s -()- Paper:p;
              PRINT res[res.id];
            }
            INSTALL QUERY get_papers_of_author""")

'Using graph \'OGB_MAG\'\nWarning in a transformed block of query get_papers_of_author (WARN-2): line 5, col 31\nunsatisfiable pattern _\nSuccessfully created queries: [get_papers_of_author].\nStart installing queries, about 1 minute ...\nget_papers_of_author query: curl -X GET \'https://127.0.0.1:9000/query/OGB_MAG/get_papers_of_author?auth=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

### Test the Queries with the Agent
Since the **GenerateFunction** tool automatically pulls the queries available to it at execution time, we can immediately test if our new queries are used by the agent.

In [21]:
agent.run("What institutions did author 45 work at?")



> Entering new AgentExecutor chain...
 I need to find the institutions associated with author 45
Action: MapQuestionToSchema
Action Input: What institutions did author 45 work at?
Observation:  What Institutions are IS_AFFILIATED_WITH Edges connected to Author Vertex 45?
Thought: I need to generate a function to get the institutions associated with author 45
Action: GenerateFunction
Action Input: Get the IS_AFFILIATED_WITH Edges connected to Author Vertex 45
Observation: runInstalledQuery('author_at_institutions', {'auth_id': 45})
Thought: I need to execute the function to get the institutions associated with author 45
Action: ExecuteFunction
Action Input: runInstalledQuery('author_at_institutions', {'auth_id': 45})
Observation: [{'res': [{'v_id': '649', 'v_type': 'Institution', 'attributes': {'res.id': 649}}]}]
Thought: I now know the final answer
Final Answer: The institution with id 649 is the institution that author 45 is affiliated with.

> Finished chain.


'The institution with id 649 is the institution that author 45 is affiliated with.'

In [22]:
agent.run("What papers did author 5 write?")



> Entering new AgentExecutor chain...
 I need to find the papers written by author 5
Action: MapQuestionToSchema
Action Input: What papers did author 5 write?
Observation:  What WROTE Edges are there from Author Vertex 5?
Thought: I need to generate a function to get the papers written by author 5
Action: GenerateFunction
Action Input: Get WROTE Edges from Author Vertex 5
Observation: runInstalledQuery('get_papers_of_author', {'auth': '5'})
Thought: I need to execute the function to get the papers written by author 5
Action: ExecuteFunction
Action Input: runInstalledQuery('get_papers_of_author', {'auth': '5'})
Observation: [{'res': [{'v_id': '539403', 'v_type': 'Paper', 'attributes': {'res.id': 539403}}, {'v_id': '676656', 'v_type': 'Paper', 'attributes': {'res.id': 676656}}, {'v_id': '215835', 'v_type': 'Paper', 'attributes': {'res.id': 215835}}]}]
Thought: I now know the final answer
Final Answer: The papers written by author 5 are 539403, 676656, and 215835.

> Finished chain.


'The papers written by author 5 are 539403, 676656, and 215835.'

## Add Graph Data Science Capabilities

One of the questions that the agent failed to answer was "What is the most influential paper based on citations?". This is a question that the *PageRank algorithm* can answer. Here, we are going to install the TigerGraph implementation of PageRank to make it available to the agent.

In [23]:
f = conn.gds.featurizer()

f.installAlgorithm("tg_pagerank")

Installing and optimizing the queries, it might take a minute...
Queries installed successfully


'tg_pagerank'

### Test the Question
Here, the agent maps the question of "influence" to the available PageRank algorithm, and correctly fills in the parameters.

In [24]:
agent.run("By citations, what are the 10 most influential papers?")



> Entering new AgentExecutor chain...
 I need to find the papers with the most citations
Action: MapQuestionToSchema
Action Input: By citations, what are the 10 most influential papers?
Observation:  By CITES Edges, what are the 10 most influential Paper Vertices?
Thought: I need to generate a function to get the most influential papers
Action: GenerateFunction
Action Input: By CITES Edges, what are the 10 most influential Paper Vertices?
Observation: runInstalledQuery('tg_pagerank', {'v_type': 'Paper', 'e_type': 'CITES', 'top_k': 10})
Thought: I need to execute the function to get the most influential papers
Action: ExecuteFunction
Action Input: runInstalledQuery('tg_pagerank', {'v_type': 'Paper', 'e_type': 'CITES', 'top_k': 10})
Observation: [{'@@top_scores_heap': [{'Vertex_ID': '428523', 'score': 392.87326}, {'Vertex_ID': '384889', 'score': 251.80214}, {'Vertex_ID': '377502', 'score': 149.10182}, {'Vertex_ID': '61855', 'score': 129.7406}, {'Vertex_ID': '416200', 'score': 129.22864

'The 10 most influential papers, by citations, are 428523, 384889, 377502, 61855, 416200, 380836, 40415, 437914, 394406, and 533629.'

## Create Specific Tools
There is one more question that originally failed that we have not yet resolved - the question of "How many Papers have a y attribute equal to 1?". To do this, we could write a GSQL query. However, the pyTigerGraph `getVertexCount()` function has a `where` parameter that can be used to accomplish the same task. Unfortunately, the agent doesn't know this due to the lack of specific knowledge about pyTigerGraph. To resolve this, we can define a custom tool for getting the vertex count.

In [25]:
class GenerateVertexCountFunction(BaseTool):
    name = "GenerateVertexCountFunction"
    description = "Generate a pyTigerGraph function call to count vertices with filters."
    conn: TigerGraphConnection = None
    llm: LLM = None
    handle_tool_error: bool =True
    
    def __init__(self, conn, llm):
        super().__init__()
        self.conn = conn
        self.llm = llm
    
    def _run(self, question: str) -> str:
        prompt_template = """Use the vertex types and any potential filters to generate the conn.getVertexCount() function call.
                          """+self.conn.getVertexCount.__doc__.replace("{", "").replace("}", "")+"""
                            Vertex Types: {vertices}
                            Question: {question}
                            Python Call to run the function: conn."""
        
        
        PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["question", "vertices"]
        )
        queries = conn.getInstalledQueries()
        inputs = [{"question": question, 
                    "vertices": self.conn.getVertexTypes()
                  }]
        
        chain = LLMChain(llm=self.llm, prompt=PROMPT)
        generated = chain.apply(inputs)[0]["text"]
        return "Use in ExecuteFunction tool: "+generated
    
    async def _arun(self) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [26]:
tools = [MapQuestionToSchema(conn, llm), GenerateFunction(conn, llm), GenerateVertexCountFunction(conn, llm), ExecuteFunction(conn)]

In [27]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [28]:
agent.run("How many papers have a y attribute equal to 1?")



> Entering new AgentExecutor chain...
 I need to count the number of papers with a y attribute equal to 1.
Action: MapQuestionToSchema
Action Input: How many papers have a y attribute equal to 1?
Observation:  How many Paper Vertexes have a y attribute equal to 1?
Thought: I need to generate a function to count the number of papers with a y attribute equal to 1.
Action: GenerateVertexCountFunction
Action Input: Paper Vertex, y attribute equal to 1
Observation: Use in ExecuteFunction tool: getVertexCount('Paper', where='y=1')
Thought: I need to execute the function to get the answer.
Action: ExecuteFunction
Action Input: getVertexCount('Paper', where='y=1')
Observation: 30902
Thought: I now know the final answer.
Final Answer: 30902 papers have a y attribute equal to 1.

> Finished chain.


'30902 papers have a y attribute equal to 1.'

# Conclusion

Integrating TigerGraph with LangChain using pyTigerGraph opens up exciting possibilities for enhancing natural language interfaces for graph databases. By combining the power of LLMs with the flexibility of graph databases, users can interact with the data using everyday language, reducing the learning curve and enabling more intuitive data exploration. With the LangChain AI Agent, users can ask questions, receive meaningful responses, and gain insights from the graph database in a seamless and user-friendly manner. These agents can be used by business analysts for internal uses, or can serve as the foundation for chatbots to serve results to customers.